In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate

In [5]:
# BLOCK: SAMPLING DATA & TRAINING NCF MODEL
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot

# ====================================================
# BƯỚC 1: LOAD VÀ LỌC DỮ LIỆU (QUAN TRỌNG)
# ====================================================
print("⏳ Đang đọc file lịch sử nghe nhạc (sẽ mất khoảng 1-2 phút)...")

# 1. Load file Vector Database (để lấy danh sách bài hát hợp lệ)
df_meta = pd.read_csv('music_vector_database.csv')
valid_track_ids = set(df_meta['track_id'].unique())
print(f"✅ Đã load danh sách bài hát hợp lệ: {len(valid_track_ids)} bài.")

# 2. Load file User History (Dùng chunksize để tránh tràn RAM nếu file quá lớn)
# Đọc file lịch sử nghe nhạc
# Lưu ý: Thay 'User Listening History.csv' bằng tên file thật của bạn nếu khác
try:
    df_history = pd.read_csv('User Listening History.csv')
except:
    print("⚠️ Không tìm thấy file csv, hãy upload file User Listening History.csv lên Colab!")
    # Stop code here nếu lỗi

print(f"📊 Dữ liệu gốc: {len(df_history):,} dòng.")

# 3. LỌC DỮ LIỆU: Chỉ lấy bài hát CÓ trong kho nhạc của chúng ta
# Bước này cực quan trọng để 2 model khớp nhau
df_clean = df_history[df_history['track_id'].isin(valid_track_ids)].copy()
print(f"🧹 Sau khi lọc bài hát không tồn tại: còn {len(df_clean):,} dòng.")

⏳ Đang đọc file lịch sử nghe nhạc (sẽ mất khoảng 1-2 phút)...
✅ Đã load danh sách bài hát hợp lệ: 50683 bài.
📊 Dữ liệu gốc: 9,711,301 dòng.
🧹 Sau khi lọc bài hát không tồn tại: còn 9,711,301 dòng.


In [ ]:
# ====================================================
# BƯỚC 2: THỰC HIỆN CÁCH 2 - SAMPLING (LẤY MẪU)
# ====================================================
# Tỷ lệ lấy mẫu: 0.1 nghĩa là 10% (Nếu file gốc 10tr dòng -> lấy 1tr dòng)
# Bạn có thể chỉnh frac=0.2 nếu muốn nhiều dữ liệu hơn
SAMPLE_RATIO = 0.1

print(f"🎲 Đang lấy mẫu ngẫu nhiên {SAMPLE_RATIO*100}% dữ liệu...")
df_sampled = df_clean.sample(frac=SAMPLE_RATIO, random_state=42)

print(f"✅ DỮ LIỆU DÙNG ĐỂ TRAIN: {len(df_sampled):,} dòng.")
print("-" * 50)

🎲 Đang lấy mẫu ngẫu nhiên 10.0% dữ liệu...
✅ DỮ LIỆU DÙNG ĐỂ TRAIN: 971,130 dòng.
--------------------------------------------------


In [7]:
print("⚙️ Đang mã hóa User ID và Track ID...")

# Mã hóa User ID sang số nguyên (0, 1, 2...)
user_encoder = LabelEncoder()
df_sampled['user_idx'] = user_encoder.fit_transform(df_sampled['user_id'])

# Mã hóa Track ID sang số nguyên
track_encoder = LabelEncoder()
df_sampled['track_idx'] = track_encoder.fit_transform(df_sampled['track_id'])

# Lấy số lượng user và track để khai báo cho Model
num_users = df_sampled['user_idx'].nunique()
num_tracks = df_sampled['track_idx'].nunique()

print(f"   - Số lượng Users: {num_users:,}")
print(f"   - Số lượng Tracks: {num_tracks:,}")

# Chuẩn bị Input (X) và Output (y)
# X: Cặp [User_Index, Track_Index]
X = df_sampled[['user_idx', 'track_idx']].values

# y: Play count (Đã chuẩn hóa)
# Dùng logarit để giảm chênh lệch giữa người nghe 1 lần và 1000 lần
df_sampled['rating'] = np.log1p(df_sampled['playcount']) 
max_rating = df_sampled['rating'].max()
y = df_sampled['rating'].values / max_rating # Scale về khoảng 0-1

# Chia tập Train/Test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

⚙️ Đang mã hóa User ID và Track ID...
   - Số lượng Users: 458,232
   - Số lượng Tracks: 25,400


In [8]:
# ====================================================
# BƯỚC 4: XÂY DỰNG & TRAIN MODEL NCF
# ====================================================
print("\n🚀 BẮT ĐẦU TRAIN MODEL...")

EMBEDDING_SIZE = 32 # Kích thước vector ẩn

# Input Layers
user_input = Input(shape=(1,), name='user_input')
track_input = Input(shape=(1,), name='track_input')

# Embedding Layers
user_emb = Embedding(num_users, EMBEDDING_SIZE, name='user_embedding')(user_input)
track_emb = Embedding(num_tracks, EMBEDDING_SIZE, name='track_embedding')(track_input)

# Flatten
user_vec = Flatten()(user_emb)
track_vec = Flatten()(track_emb)

# Dot Product (Tính độ tương đồng giữa User và Track)
output = Dot(axes=1)([user_vec, track_vec])

# Model
model = Model(inputs=[user_input, track_input], outputs=output)
model.compile(optimizer='adam', loss='mse') # Mean Squared Error

# Train
history = model.fit(
    x=[X_train[:, 0], X_train[:, 1]], 
    y=y_train,
    batch_size=1024, # Batch lớn để chạy nhanh
    epochs=5,        # 5 vòng là đủ cho demo
    validation_data=([X_test[:, 0], X_test[:, 1]], y_test),
    verbose=1
)


🚀 BẮT ĐẦU TRAIN MODEL...
Epoch 1/5
759/759 ━━━━━━━━━━━━━━━━━━━━ 63s 83ms/step - loss: 0.0272 - val_loss: 0.0271
Epoch 2/5
759/759 ━━━━━━━━━━━━━━━━━━━━ 58s 76ms/step - loss: 0.0201 - val_loss: 0.0269
Epoch 3/5
759/759 ━━━━━━━━━━━━━━━━━━━━ 58s 76ms/step - loss: 0.0071 - val_loss: 0.0269
Epoch 4/5
759/759 ━━━━━━━━━━━━━━━━━━━━ 57s 76ms/step - loss: 0.0022 - val_loss: 0.0270
Epoch 5/5
759/759 ━━━━━━━━━━━━━━━━━━━━ 59s 77ms/step - loss: 0.0012 - val_loss: 0.0270


In [9]:
# ====================================================
# BƯỚC 5: LƯU KẾT QUẢ
# ====================================================
print("\n💾 Đang lưu model và encoder...")
model.save('ncf_model_sampled.h5')

# Lưu Encoder để sau này dùng lại (Map từ ID thật -> ID số)
import pickle
with open('user_encoder.pkl', 'wb') as f: pickle.dump(user_encoder, f)
with open('track_encoder.pkl', 'wb') as f: pickle.dump(track_encoder, f)

print("✅ HOÀN TẤT! Bạn đã có file 'ncf_model_sampled.h5' để chạy hệ thống Hybrid.")


💾 Đang lưu model và encoder...
✅ HOÀN TẤT! Bạn đã có file 'ncf_model_sampled.h5' để chạy hệ thống Hybrid.
